###### Content under Creative Commons Attribution license CC-BY 4.0, code under BSD 3-Clause License © 2018  by D. Koehn, notebook style sheet by L.A. Barba, N.C. Clementi

###### Translated by Huizhe Di @ SCSIO

# 求解各向同性的弹性二维SH问题通过交错网格有限差分法


用声波方程模拟地震波在固体地球中的传播是一种非常粗略的近似。它不仅不能描述S体波或者Rayleigh/Love面波，同样也不能描述任何PS或SP的转换。

最简单的弹性学近似就是二维SH问题，它可以用偏微分方程来描述，在数学上与二维声学问题非常相似。让我们对二维SH问题进行一个简短的回顾。

## 二维SH波传播的运动方程在各向同性弹性介质

就像在 [这节课](http://nbviewer.jupyter.org/github/daniel-koehn/Theory-of-seismic-waves-II/blob/master/01_Analytical_solutions/4_2D_1D_elastic_acoustic_approx.ipynb)讲的, 将三维各向同性弹性问题分解解耦为两个问题，即二维PSV问题和二维SH问题。 假设垂直于x-z平面的y方向上只有非零粒子位移，这就引出了 **二维 SH 问题**, x表示水平距离，z表示深度, 波的传播可以用下面的偏微分方程组来描述:

\begin{align}
\rho \frac{\partial^2 u_y}{\partial t^2} &= \frac{\partial \sigma_{yx}}{\partial x} + \frac{\partial \sigma_{yz}}{\partial z} + f_y, \notag\\
\sigma_{yx} &= \mu \frac{\partial u_{y}}{\partial x},\\
\sigma_{yz} &= \mu \frac{\partial u_{y}}{\partial z}, \notag\\
\end{align}

这里， $\rho$ 为密度，$\mu$ 为剪切模量，$\rm{u_y}$ 为y方向上的粒子位移，$\sigma_{yx}$, $\rm{\sigma_{yz}}$ 为剪应力分量, $f_y$ 为y方向的有向体力。 

我们可以修改这个问题通过引入 **粒子速度** $v_y = \frac{\partial u_y}{\partial t}$. 求应力应变关系的一阶时间导数在等式(1)中，假设剪切模量不随时间变化，给出耦合的一阶偏微分方程组:

\begin{align}
\rho \frac{\partial v_y}{\partial t} &= \frac{\partial \sigma_{yx}}{\partial x} + \frac{\partial \sigma_{yz}}{\partial z} + f_y, \notag\\
\frac{\partial \sigma_{yx}}{\partial t} &= \mu \frac{\partial v_{y}}{\partial x},\\
\frac{\partial \sigma_{yz}}{\partial t} &= \mu \frac{\partial v_{y}}{\partial z}. \notag\\
\end{align}

这被称为二维SH问题的 **应力-速度公式** 。 下面，我们通过交错网格有限差分法求解等式(2)。

## 通过交错网格有限差分法求解二维SH问题

对于之前一维和二维声波方程的有限差分近似，我们在笛卡尔网格上离散了时间和空间上的弹性运动方程。然而，相较于声学波动方程，二维SH问题包含了3个波场 $v_y,\; \sigma_{yx},\; \sigma_{yz}$ 和2个材料参数 $\rho,\; \mu$。 

为了正确地计算与波场和材料参数相关的空间导数, [Jean Virieux](https://jean-virieux.obs.ujf-grenoble.fr/) 在 [这篇文章](http://jean-virieux.obs.ujf-grenoble.fr/IMG/pdf/GPY_1984_VIRIEUX.pdf) 介绍了求解二维SH问题的 **交错网格有限差分法** :

<img src="images/SG_SH-Cart.png" width="50%">

粒子速度 $v_y$ 和密度 $\rho$ 位于网格点(i,j)上, 然而剪应力 $\sigma_{yx}$ 位于x方向的半个网格点(i+1/2,j)上，剪应力 $\sigma_{yz}$ 被转移到z方向的半个网格点(i,j+1/2)上。 此外，剪切模量必须谐波平均:

\begin{align}
\mu_{x} &= \mu(i+1/2,j) = 2 \biggl[\biggl(\mu^{-1}_{i,j}+\mu^{-1}_{i+1,j}\biggr)\biggr]^{-1} \notag\\
\mu_{z} &= \mu(i,j+1/2) = 2 \biggl[\biggl(\mu^{-1}_{i,j}+\mu^{-1}_{i,j+1}\biggr)\biggr]^{-1} \notag
\end{align}

$dx,\; dz$ 分别表示相邻的两个空间格点在x和z方向上的空间距离。下一个问题是如何近似这个交错网格上的空间偏导数。

我们从动量方程开始

\begin{equation}
\rho \frac{\partial v_y}{\partial t} = \frac{\partial \sigma_{yx}}{\partial x} + \frac{\partial \sigma_{yz}}{\partial z} + f_y. \notag
\end{equation}

我们将正确的有限差分运算符的估算分为两个步骤:

1. 位于交错网格上的方程左边上的材料参数和波场在哪里? 在动量方程中，它们被放置在网格点(i,j)上。
<img src="images/SG_SH-Cart.png" width="50%">
2. 方程右边上的所有导数都必须以第一步中估计的网格点为中心。因此，动量方程右边的空间导数可以用二阶中心有限差分算子逼近:



\begin{align}
\biggl(\frac{\partial \sigma_{yx}}{\partial x}\biggr)^c(i,j) &\approx \frac{\sigma_{yx}(i+1/2,j) - \sigma_{yx}(i-1/2,j)}{dx},\notag\\ 
\biggl(\frac{\partial \sigma_{yz}}{\partial z}\biggr)^c(i,j) &\approx \frac{\sigma_{yz}(i,j+1/2) - \sigma_{yz}(i,j-1/2)}{dz}.\notag
\end{align}

对于应力应变关系中的剪应力 $\sigma_{yx}$

\begin{equation}
\frac{\partial \sigma_{yx}}{\partial t} = \mu \frac{\partial v_{y}}{\partial x}, \notag
\end{equation}

我们使用相同的方法:

1. 等式左边的 $\sigma_{yx}$ 位于网格点(i+1/2,j)上
<img src="images/SG_SH-Cart.png" width="50%">
2. 因此，我们用二阶中心算子逼近等式右边上的空间导数:

\begin{equation}
\mu_x \biggl(\frac{\partial v_{y}}{\partial x}\biggr)^c(i+1/2,j) \approx \mu_x \frac{v_y(i+1,j) - v_{y}(i,j)}{dx},\notag 
\end{equation}

对于应力应变关系中的剪应力 $\sigma_{yz}$

\begin{equation}
\frac{\partial \sigma_{yz}}{\partial t} = \mu \frac{\partial v_{y}}{\partial z}, \notag
\end{equation}

我们得到:

1. 等式左边的 $\sigma_{yz}$ 位于网格点(i,j+1/2)上
<img src="images/SG_SH-Cart.png" width="50%">
2. 因此，我们用二阶中心算子逼近等式右边上的空间导数:

\begin{equation}
\mu_z \biggl(\frac{\partial v_{y}}{\partial z}\biggr)^c(i,j+1/2) \approx \mu_z \frac{v_y(i,j+1) - v_{y}(i,j)}{dz}.\notag 
\end{equation}


综上所述，我们得到了以下二维各向同性SH问题的空间有限差分近似:

\begin{align}
\rho \frac{\partial v_y}{\partial t} (i,j)&= \biggl(\frac{\partial \sigma_{yx}}{\partial x}\biggr)^c(i,j) + \biggl(\frac{\partial \sigma_{yz}}{\partial z}\biggl)^c(i,j) + f_y(i,j), \notag\\
\frac{\partial \sigma_{yx}}{\partial t} (i+1/2,j) &= \mu_x \biggl(\frac{\partial v_{y}}{\partial x}\biggr)^c(i+1/2,j),\notag\\
\frac{\partial \sigma_{yz}}{\partial t}(i,j+1/2) &= \mu_z \biggl(\frac{\partial v_{y}}{\partial z}\biggr)^c(i,j+1/2). \notag\\
\end{align}

最后，我们对偏微分方程左边上的时间导数进行了近似，使用 [**Leapfrog method**](https://en.wikipedia.org/wiki/Leapfrog_integration) 这是基于交错时间网格在时间步长 n 和 n+1/2:

\begin{align}
\scriptsize\rho \frac{v_y(i,j,n+1/2)-v_y(i,j,n-1/2)}{dt}&\scriptsize= \biggl(\frac{\partial \sigma_{yx}}{\partial x}\biggr)^c(i,j,n) + \biggl(\frac{\partial \sigma_{yz}}{\partial z}\biggl)^c(i,j,n) + f_y(i,j,n), \notag\\
\scriptsize\frac{\sigma_{yx}(i+1/2,j,n+1)-\sigma_{yx}(i+1/2,j,n)}{dt} &\scriptsize= \mu_x \biggl(\frac{\partial v_{y}}{\partial x}\biggr)^c(i+1/2,j,n+1/2),\notag\\
\scriptsize\frac{\sigma_{yz}(i,j+1/2,n+1)-\sigma_{yz}(i,j+1/2,n)}{dt} &\scriptsize= \mu_z \biggl(\frac{\partial v_{y}}{\partial z}\biggr)^c(i,j+1/2,n+1/2). \notag\\
\end{align}

重新排列时间步长 n+1/2 和 n+1 到等式左边，在交错网格上得到最终的二维SH问题的有限差分方案:

\begin{align}
\scriptsize 
v_y(i,j,n+1/2) &\scriptsize= \frac{dt}{\rho}\biggl\{\biggl(\frac{\partial \sigma_{yx}}{\partial x}\biggr)^c(i,j,n) + \biggl(\frac{\partial \sigma_{yz}}{\partial z}\biggl)^c(i,j,n) + f_y(i,j,n)\biggr\} + v_y(i,j,n-1/2), \notag\\
\scriptsize\sigma_{yx}(i+1/2,j,n+1) &\scriptsize= dt\; \mu_x \biggl(\frac{\partial v_{y}}{\partial x}\biggr)^c(i+1/2,j,n+1/2) + \sigma_{yx}(i+1/2,j,n),\notag\\
\scriptsize\sigma_{yz}(i,j+1/2,n+1) &\scriptsize= dt\; \mu_z \biggl(\frac{\partial v_{y}}{\partial z}\biggr)^c(i,j+1/2,n+1/2) + \sigma_{yz}(i,j+1/2,n), \notag\\
\end{align}

我们将在下节课中实现它。

## 我们学习到:

- 如何用交错网格有限差分法求解二维SH问题。